In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
from scipy import stats
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
import matplotlib.pyplot as plt

from lavian_et_al_2025.imaging.imaging_classes import TwoPExperiment
from lavian_et_al_2025.data_location import master_landmarks

from pathlib import Path
from scipy import stats

In [ ]:
master_ablated = master_landmarks / "2p - ablations" / 'ntr'
master_control = master_landmarks / "2p - ablations" / 'control'

fish_list_control_pre = list(master_control.glob("*_pre*"))
fish_list_ablated_pre = list(master_ablated.glob("*_pre*"))

fish_list_control_post = list(master_control.glob("*_post*"))
fish_list_ablated_post = list(master_ablated.glob("*_post*"))


In [ ]:
num_ntr = len(fish_list_ablated_pre)
num_control = len(fish_list_control_pre)

print(num_ntr)
print(num_control)

In [ ]:
count = 0

for fish in fish_list_ablated_pre:
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        if count == 0:
            all_reg_values_ntr_pre = sens_regs
            count = 1
        else:
            all_reg_values_ntr_pre = np.append(all_reg_values_ntr_pre, sens_regs)

In [ ]:
count = 0

for fish in fish_list_control_pre:
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        if count == 0:
            all_reg_values_control_pre = sens_regs
            count = 1
        else:
            all_reg_values_control_pre = np.append(all_reg_values_control_pre, sens_regs)

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(6,6), sharex=False, sharey=False)

ax[0,0].set_title('NTR (n=8)')
ax[0,0].hist(all_reg_values_ntr_pre, bins=100, alpha=1, density=True, histtype='step', color='cornflowerblue')
ax[0,0].set_xlim(-0.7, 0.7)
ax[1,0].set_xlim(-0.7, 0.7)
ax[0,0].spines['right'].set_visible(False)
ax[0,0].spines['top'].set_visible(False)

ax[1,0].set_xlabel('Correlation with all regressors')
ax[0,0].set_ylabel('Density')
ax[1,0].set_ylabel('Density')

ax[1,0].set_title('Control (n=8)')
ax[1,0].hist(all_reg_values_control_pre, bins=100, alpha=1, density=True, histtype='step', color='cornflowerblue')
ax[1,0].spines['right'].set_visible(False)
ax[1,0].spines['top'].set_visible(False)

In [ ]:
fig.subplots_adjust(hspace=0.3, wspace=0.3)

In [ ]:
count = 0

for fish in fish_list_ablated_post:
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        if count == 0:
            all_reg_values_ntr_post = sens_regs
            count = 1
        else:
            all_reg_values_ntr_post = np.append(all_reg_values_ntr_post, sens_regs)

In [ ]:
count = 0

for fish in fish_list_control_post:
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        if count == 0:
            all_reg_values_control_post = sens_regs
            count = 1
        else:
            all_reg_values_control_post = np.append(all_reg_values_control_post, sens_regs)

In [ ]:
ax[0,0].hist(all_reg_values_ntr_post, bins=100, alpha=1, density=True, histtype='step', color='mediumorchid')
ax[0,0].set_xlim(-0.5, 0.5)
ax[0,0].spines['right'].set_visible(False)
ax[0,0].spines['top'].set_visible(False)


ax[1,0].hist(all_reg_values_control_post, bins=100, alpha=1, density=True, histtype='step', color='mediumorchid')
ax[1,0].set_xlim(-0.5, 0.5)
ax[1,0].spines['right'].set_visible(False)
ax[1,0].spines['top'].set_visible(False)

In [ ]:
# Calculate the F-statistic
f = np.var(all_reg_values_ntr_pre, ddof=1) / np.var(all_reg_values_ntr_post, ddof=1)

# Calculate degrees of freedom
dfn = len(all_reg_values_ntr_pre) - 1  # degrees of freedom numerator
dfd = len(all_reg_values_ntr_post) - 1  # degrees of freedom denominator

# Perform the F-test
p_value = stats.f.cdf(f, dfn, dfd)
p_value = 2 * min(p_value, 1 - p_value)  # two-tailed test

print(f"F-statistic: {f}")
print(f"p-value: {p_value}")

In [ ]:
# Calculate the F-statistic
f = np.var(all_reg_values_control_pre, ddof=1) / np.var(all_reg_values_control_post, ddof=1)

# Calculate degrees of freedom
dfn = len(all_reg_values_ntr_pre) - 1  # degrees of freedom numerator
dfd = len(all_reg_values_ntr_post) - 1  # degrees of freedom denominator

# Perform the F-test
p_value = stats.f.cdf(f, dfn, dfd)
p_value = 2 * min(p_value, 1 - p_value)  # two-tailed test

print(f"F-statistic: {f}")
print(f"p-value: {p_value}")

In [ ]:
count = 0
thresh = 0.3
num_corr_cells_ntr = np.zeros((2, num_ntr))

for fish in fish_list_ablated_pre:
    print(fish)
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        num_corr_cells_ntr[0, count] += np.shape(np.where(np.abs(sens_regs) > thresh)[0])[0]
        
    count += 1

In [ ]:
count = 0

for fish in fish_list_ablated_post:
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        num_corr_cells_ntr[1, count] += np.shape(np.where(np.abs(sens_regs) > thresh)[0])[0]
        
    count += 1

In [ ]:
count = 0
num_corr_cells_control = np.zeros((2, num_ntr))

for fish in fish_list_control_pre:
    print(fish)
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        num_corr_cells_control[0, count] += np.shape(np.where(np.abs(sens_regs) > thresh)[0])[0]
        
    count += 1

In [ ]:
count = 0

for fish in fish_list_control_post:
    suite2p_path = fish / 'suite2p'
    path_list = list(suite2p_path.glob("*00*"))
    
    num_planes = len(path_list)
    
    for path in path_list:
        sens_regs = fl.load(path / 'sensory_regressors_cells.h5')['regressors_values']
        
        #apply mask
        ipn_coords = fl.load(path / 'ipn_coords.h5')['coords_to_keep']
        sens_regs = sens_regs[:, ipn_coords]
        
        num_corr_cells_control[1, count] += np.shape(np.where(np.abs(sens_regs) > thresh)[0])[0]
        
    count += 1

In [ ]:
ax[0,1].plot(num_corr_cells_ntr, color='gray')
ax[0,1].spines['right'].set_visible(False)
ax[0,1].spines['top'].set_visible(False)
ax[0,1].set_xlim(-0.5, 1.5)

ax[1,1].plot(num_corr_cells_control, color='gray')
ax[1,1].spines['right'].set_visible(False)
ax[1,1].spines['top'].set_visible(False)
ax[1,1].set_xlim(-0.5, 1.5)
ax[1,1].set_ylim(0, 150)
ax[0,1].set_ylim(0, 150)

In [ ]:
ax[0,1].errorbar([0, 1], np.nanmean(num_corr_cells_ntr, axis=1), np.nanstd(num_corr_cells_ntr, axis=1) / 8, color='k')
ax[1,1].errorbar([0, 1], np.nanmean(num_corr_cells_control, axis=1), np.nanmean(num_corr_cells_control, axis=1) / 8, color='k')

In [ ]:
############### t-test for right panels 
stats.ttest_rel(num_corr_cells_ntr[0], num_corr_cells_ntr[1], alternative='greater')

In [ ]:
stats.ttest_rel(num_corr_cells_control[0], num_corr_cells_control[1], alternative='two-sided')

In [ ]:
file_name = "Regressors corr distribution density n=" + str(num_ntr) + " ipn coords.pdf"
fig.savefig(master_ablated / file_name, dpi=300)